In [19]:
import torchmetrics as tm
from typing import Union
from openvino.runtime.ie_api import CompiledModel
from tqdm import tqdm
from pathlib import Path
from openvino.tools import mo
from openvino.runtime import serialize

import torch
import nncf
from openvino.runtime import Core

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


In [2]:
from utils import *

In [3]:
MODEL_DIR = "./model/"

In [11]:
class PetModel(nn.Module):
    def __init__(self, arch, encoder_name, in_channels, out_classes, **kwargs):
        super().__init__()
        self.model = smp.create_model(
            arch, encoder_name=encoder_name, in_channels=in_channels, classes=out_classes, **kwargs
        )

        params = smp.encoders.get_preprocessing_params(encoder_name)
        self.std = torch.tensor(params["std"]).view(1, 3, 1, 1)
        self.mean = torch.tensor(params["mean"]).view(1, 3, 1, 1)
        self.loss_fn = smp.losses.DiceLoss(
            smp.losses.BINARY_MODE, from_logits=True)

    def forward(self, image):
        image = (image - self.mean) / self.std
        mask = self.model(image)
        return mask


In [12]:
model = torch.load("./model/model.pth", map_location=torch.device('cpu'))

In [6]:
train_dataloader, valid_dataloader, test_dataloader = give_data_loaders()

In [7]:
def compute_dice(model: Union[torch.nn.Module, CompiledModel], dataset):
    metric = tm.Dice(zero_division = 0, num_classes = 2)
    with torch.no_grad():
        for batch in tqdm(dataset, total = len(dataset)):
            image = batch["image"]
            target = batch["mask"]
            input_image = torch.as_tensor(image) #.unsqueeze(0)
            if isinstance(model, CompiledModel):
                output_layer = model.output(0)
                output = model(input_image)[output_layer]
                output = torch.from_numpy(output)
            else:
                output = model(input_image)
            label = torch.as_tensor(target.squeeze()).long()
            prediction = torch.sigmoid(output.squeeze()).round().long()
            metric.update(label.flatten(), prediction.flatten())
    return metric.compute()

In [7]:
fp32_dice = compute_dice(model, test_dataloader)
print(f"FP32 Dice: {fp32_dice:.3f}")

100%|██████████| 230/230 [03:37<00:00,  1.06it/s]

FP32 Dice: 0.954


In [8]:
batch = next(iter(test_dataloader))

In [16]:
batch["image"].shape

torch.Size([1, 3, 256, 256])

/home/venom/miniconda3/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:474: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn("Conv BatchNorm folding failed during the optimize process.")
/home/venom/miniconda3/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:479: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn("Linear BatchNorm folding failed during the optimize process.")
/home/venom/miniconda3/lib/python3.9/site-packages/segmentation_models_pytorch/base/model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:


In [13]:
# Save model onnx
torch.onnx.export(model, batch['image'], "model.onnx", export_params=True, opset_version=14)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [18]:
!mo --input_model ./model/model.onnx

[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.0/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/venom/repo/Segmentation_OneAPI/model.xml
[ SUCCESS ] BIN file: /home/venom/repo/Segmentation_OneAPI/model.bin


In [21]:
def transform_fn(data_item):
    """
    Extract the model's input from the data item.
    The data item here is the data item that is returned from the data source per iteration.
    This function should be passed when the data item cannot be used as model's input.
    """
    images = data_item["image"]
    return images

In [22]:
calibration_dataset = nncf.Dataset(valid_dataloader, transform_fn)
quantized_model = nncf.quantize(
    model,
    calibration_dataset,
)

INFO:nncf:Collecting tensor statistics |█               | 33 / 300
INFO:nncf:Collecting tensor statistics |███             | 66 / 300
INFO:nncf:Collecting tensor statistics |█████           | 99 / 300
INFO:nncf:Collecting tensor statistics |███████         | 132 / 300
INFO:nncf:Collecting tensor statistics |████████        | 165 / 300
INFO:nncf:Collecting tensor statistics |██████████      | 198 / 300
INFO:nncf:Collecting tensor statistics |████████████    | 231 / 300
INFO:nncf:Collecting tensor statistics |██████████████  | 264 / 300
INFO:nncf:Collecting tensor statistics |███████████████ | 297 / 300
INFO:nncf:Collecting tensor statistics |████████████████| 300 / 300
INFO:nncf:Compiling and loading torch extension: quantized_functions_cpu...
INFO:nncf:Finished loading torch extension: quantized_functions_cpu
INFO:nncf:BatchNorm statistics adaptation |█               | 33 / 300
INFO:nncf:BatchNorm statistics adaptation |███             | 66 / 300
INFO:nncf:BatchNorm statistics adaptati

In [25]:
dummy_input = torch.randn(1, 3, 256, 256)
int8_onnx_path = f"{MODEL_DIR}model_int8.onnx"
int8_ir_path = Path(int8_onnx_path).with_suffix(".xml")
torch.onnx.export(quantized_model, dummy_input, int8_onnx_path)
int8_ir_model = mo.convert_model(input_model=int8_onnx_path)
serialize(int8_ir_model, str(int8_ir_path))

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [27]:
core = Core()

int8_compiled_model = core.compile_model(int8_ir_model)
int8_die = compute_dice(int8_compiled_model, test_dataloader)

print(f"INT8 Dice: {int8_die:.3f}")

100%|██████████| 3669/3669 [01:14<00:00, 49.30it/s]

INT8 F1: 0.950
